In [1]:
import os, sys, importlib
import rasterio

import pandas as pd
import geopandas as gpd

from shapely.geometry import box

sys.path.append("../")

import infrasap.rasterMisc as rMisc 

# Zonal Stats on VIIRS

In [11]:
port_file = '/home/wb411133/data/Global/INFRA/PORTS/TRAVEL_TIMES/COMBO_all_ports.shp'
zonal_ntl_res = "/home/wb411133/temp/All_Port_VIIRS_ZONAL_Results.csv"
zonal_pop_area_res = "/home/wb411133/temp/All_Port_Pop_Area_Results.csv"
viirs_folder = "/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED"
viirs_vrts = []
for root, dirs, files in os.walk(viirs_folder):
    for f in files:
        if f[-4:] == ".vrt":
            viirs_vrts.append(os.path.join(root, f))

inD = gpd.read_file(port_file)
inD.head()

,threshold,IDX,PORT,geometry
0,360.0,1,12,(POLYGON ((88.93333333333334 25.84166666666667...
1,720.0,1,12,"(POLYGON ((88.49166666666667 27.4, 88.49166666..."
2,1440.0,1,12,"(POLYGON ((91.83333333333331 33.15, 91.8333333..."
3,2880.0,1,12,(POLYGON ((115.3416666666666 27.74166666666667...
4,360.0,1,19,(POLYGON ((-37.15000000000001 -4.9416666666666...


In [20]:
if not os.path.exists(zonal_ntl_res):
    all_res = {}
    for vrt in viirs_vrts:
        print(vrt)
        yr = os.path.basename(vrt).split("_")[0]
        res = rMisc.zonalStats(port_file, vrt)
        res = pd.DataFrame(res, columns=["YR_%s_%s" % (yr, x) for x in ['SUM','MIN','MAX','MEAN']])
        all_res[yr] = res

    try:
        del final
    except: 
        pass
    for key, vals in all_res.items():
        vals['IDX'] = inD['threshold']
        xx = vals.iloc[:,[0,3]]
        try:
            final = final.join(xx)
        except:
            final = xx

    

/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2012_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2013_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2014_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2015_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2016_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2017_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2018_VIIRS_annual_composite.vrt
/home/public/Data/GLOBAL/NighttimeLights/VIIRS_CORRECTED/2019_VIIRS_annual_composite.vrt


In [21]:
final['threshold'] = inD['threshold']
final['PORT'] = inD['PORT']
final.to_csv(zonal_ntl_res)

# Zonal stats on Pop

In [9]:
pop_file = "/home/public/Data/GLOBAL/Population/WorldPop_PPP_2020/ppp_2020_1km_Aggregated.tif"
res = rMisc.zonalStats(inD, pop_file, minVal=0)
res = pd.DataFrame(res, columns=['pop_SUM', 'pop_MIN', 'pop_MAX', 'pop_MEAN'])

In [12]:
res['threshold'] = inD['threshold']
res['PORT'] = inD['PORT']

# Calculate area

In [16]:
marketsheds = gpd.read_file(port_file)
marketsheds = marketsheds.to_crs({'init':'epsg:6933'})

marketsheds['AREA_km'] = marketsheds['geometry'].apply(lambda x: x.area/1000000)

In [19]:
res['AREA_km'] = marketsheds['AREA_km']
res.to_csv(zonal_pop_area_res)

# Combine Results

In [ ]:
inP = gpd.read_file(port_file)


In [ ]:
final['THRESH'] = inP['threshold']
final['PORT'] = inP['PORT']
final['WP_2020_SUM'] = res['pop_SUM']
final['AREA'] = marketsheds['AREA']

In [ ]:
final.to_csv("/home/wb411133/temp/SouthAsia_Port_Results.csv")